In [1]:
import torch
from torch.utils.data import DataLoader
from object_detection_data import TransformedDataset
import utils
from pytorch_lightning.callbacks import RichProgressBar
import pytorch_lightning as pl
from object_detection_models import MultiClassJetNet

torch.manual_seed(2)


In [2]:
# TODO: Parametrize by aspect ratio
default_scalings = utils.scalings(
    torch.tensor([1.0, 1.0]), torch.tensor([0.5, 1.0]), 6, 4
)
num_classes = 4
encoder = utils.Encoder(default_scalings, num_classes)
transformed_train_data = TransformedDataset("data/train", encoder)
transformed_val_data = TransformedDataset("data/val", encoder)


In [3]:
train_loader = DataLoader(
    transformed_train_data, batch_size=32, shuffle=True, num_workers=4
)
val_loader = DataLoader(
    transformed_val_data, batch_size=32, shuffle=False, num_workers=4
)

pl_model = MultiClassJetNet(encoder, 1e-3)
trainer = pl.Trainer(
    limit_predict_batches=100, max_epochs=200, callbacks=[RichProgressBar()]
)
trainer.fit(model=pl_model, train_dataloaders=train_loader, val_dataloaders=val_loader)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


┏━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name                ┃ Type               ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ accuracy            │ MulticlassAccuracy │      0 │
│ 1 │ input_layer         │ NormConv2dReLU     │    146 │
│ 2 │ depth_wise_backbone │ Sequential         │  4.9 K │
│ 3 │ classifier          │ Sequential         │ 20.9 K │
│ 4 │ output_layer        │ Conv2d             │  2.2 K │
└───┴─────────────────────┴────────────────────┴────────┘

Trainable params: 28.2 K                                                                                           
Non-trainable params: 0                                                                                            
Total params: 28.2 K                                                                                               
Total estimated model params size (MB): 0

2023-03-08 09:11:12.372708: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-08 09:11:12.539547: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-08 09:11:13.260249: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-08 09:11:13.260316: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

Output()

/home/jonathan/hulks/ml/venv/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:48: UserWarning: 
Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
Epoch 64/199 ━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16/249 0:00:09 • 0:02:10 1.80it/s loss: 0.155 v_num: 31 